# Модифікований метод Ньютона (мінімізація)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def func(x, y):    # -----ready-----
    """
    Функція, що шукає значення цільової функції в точці
    """
    # return (10*(x1-x2)**2+(x1-1)**2)**0.25
    # return x**2+y**2
    return 3 * (x - 24) ** 2 + y * x + 7 * y ** 2

### Перші, другі похідні (Права, Ліва, Центральна схеми)

In [ ]:

def derivative1(x, y, h, which):     # ----ready-----
    """
    Функція для обчислення ПЕРШОЇ похідної за ЦЕНТРАЛЬНОЮ схемою
    """
    if which == "x":
        return (func(x + h, y) - func(x - h, y)) / (2 * h)
    else:
        return (func(x, y + h) - func(x, y - h)) / (2 * h)


def derivative2(x, y, h, which):     # ----ready-----
    """
    Функція для обчислення ПЕРШОЇ похідної за ЛІВОЮ схемою
    """
    if which == "x":
        return (func(x, y) - func(x - h, y)) / h
    else:
        return (func(x, y) - func(x, y - h)) / h


def derivative3(x, y, h, which):       # ----ready-----
    """
    Функція для обчислення ПЕРШОЇ похідної за ПРАВОЮ схемою
    """
    if which == "x":
        return (func(x + h, y) - func(x, y)) / h
    else:
        return (func(x, y + h) - func(x, y)) / h


def derivative1_2(x, y, h, which):       # ----ready-----
    """
    Функція для обчислення ДРУГОЇ похідної за ЦЕНТРАЛЬНОЮ схемою
    """
    if which == "xx":
        return (func(x + 2 * h, y) - 2 * func(x, y) + func(x - 2 * h, y)) / (4 * h ** 2)
    elif which == "yy":
        return (func(x, y + 2 * h) - 2 * func(x, y) + func(x, y - 2 * h)) / (4 * h ** 2)
    else:
        return (func(x + h, y + h) - func(x + h, y - h) - func(x - h, y + h) + func(x - h, y - h)) / (4 * h ** 2)


def derivative2_2(x, y, h, which):       # ----ready-----
    """
    Функція для обчислення ДРУГОЇ похідної за ЛІВОЮ схемою
    """
    if which == "xx":
        return (func(x, y) - 2 * func(x - h, y) + func(x - 2 * h, y)) / (h ** 2)
    elif which == "yy":
        return (func(x, y) - 2 * func(x, y - h) + func(x, y - 2 * h)) / (h ** 2)
    else:
        return (func(x, y) - func(x, y - h) - func(x - h, y) + func(x - h, y - h)) / (h ** 2)


def derivative3_2(x, y, h, which):       # ----ready-----
    """
    Функція для обчислення ДРУГОЇ похідної за ПРАВОЮ схемою
    """
    if which == "xx":
        return (func(x + 2 * h, y) - 2 * func(x + h, y) + func(x, y)) / (h ** 2)
    elif which == "yy":
        return (func(x, y + 2 * h) - 2 * func(x, y + h) + func(x, y)) / (h ** 2)
    else:
        return (func(x + h, y + h) - func(x + h, y) - func(x, y + h) + func(x, y)) / (h ** 2)

### Критерії закінчення

In [ ]:
def check_criterium1(xk, yk, xk_plus_1, yk_plus_1, epsilon):   # works
    """
    Функція для перевірки виконання першого критерія закінчення
    """
    if ((((abs(xk_plus_1) - abs(xk)) ** 2 + (abs(yk_plus_1) - abs(yk)) ** 2) ** 0.5) / ((xk ** 2 + yk ** 2) ** 0.5)) \
            <= epsilon and \
            abs(func(xk_plus_1, yk_plus_1) - func(xk, yk)) / func(xk, yk) <= epsilon:
        return True
    else:
        return False


def check_criterium2(xk1, xk2, epsilon):   # works
    """
    Функція для перевірки виконання другого критерія закінчення
    """
    grad = gradient(xk1, xk2)
    if (grad[0] ** 2 + grad[1] ** 2) ** 0.5 <= epsilon:
        return True
    else:
        print((grad[0] ** 2 + grad[1] ** 2) ** 0.5)
        return False

### Вектор-градієнт функції

In [ ]:
def gradient(x, y):   # works
    """
    Функція для пошуку градієнта функції в точці (x, y)
    """
    return np.array([derivative1(x, y, 0.0001, "x"), derivative1(x, y, 0.0001, "y")])

### Матриця Гессе

In [ ]:
def hessian(x, y):   # works
    """
    Функція для пошуку Гeссіана функції в точці (x, y)
    """
    return np.array([[derivative1_2(x, y, 0.0001, "xx"), derivative1_2(x, y, 0.0001, "xy")],
                     [derivative1_2(x, y, 0.0001, "xy"), derivative1_2(x, y, 0.0001, "yy")]])

def routet_hessian(x, y):   # works
    """
    Функція для пошуку Оберненої матриці Гeссе функції в точці (x, y)
    """
    return np.linalg.inv(hessian(x, y))

print(hessian(33.8, 33.8), '\n')
print(routet_hessian(33.8, 33.8))

### Розмір кроку

In [ ]:
def step(x, y):   # works
    """
    Функція для пошуку розміру кроку в точці (x, y)
    """
    return np.divide((np.matmul(np.transpose(gradient(x, y)), (-gradient(x, y)))),
                     np.matmul(np.matmul(np.transpose(-gradient(x, y)), hessian(x, y)), gradient(x, y)))

### Алгоритм Свена

In [ ]:
def sven(x, s, parametr, lambd=0): #works
    """
    Функція для реалізації алгоритма Свена - пошуку інтервала невизначеності
    """
    calls_count = 0
    lambds = [lambd]
    vals = [func(x[0], x[1])]
    calls_count += 1

    delta =  parametr * np.linalg.norm(x) / np.linalg.norm(s)

    left = func(x[0] + (lambd - delta) * s[0], x[1] + (lambd - delta) * s[1])
    right = func(x[0] + (lambd + delta) * s[0], x[1] + (lambd + delta) * s[1])
    calls_count += 2
  
    if left > vals[0] and right > vals[0]:
        return lambd - delta, lambd + delta
    
    if left < right:
        sign = -1
        lambd = lambd - delta
        lambds.append(lambd)
        vals.append(left)
    else:
        sign = 1
        lambd = lambd + delta
        lambds.append(lambd)
        vals.append(right)
    
    i = 1
    done = False
    while not done:
        lambd += sign * delta * 2 ** i
        val = func(x[0] + lambd * s[0], x[1] + lambd * s[1])
        calls_count += 1
        if val >= vals[-1]:
            done = True

        lambds.append(lambd)
        vals.append(val)
        
        i += 1

    potential_min = vals[-2]
    
    lambd = (lambds[-2] + lambds[-1]) * 0.5
    val = func(x[0] + lambd * s[0], x[1] + lambd * s[1])
    calls_count += 1

    lambds.append(lambd)
    vals.append(val)
    
    print("Кількість ітерацій -", i)
    print("Кількість визовів функції -", calls_count)
    
    if val < potential_min:
        return lambds[-2], lambds[-3]
    elif val > potential_min:
        return lambds[-1], lambds[-4]

    else:
        return lambds[-1], lambds[-3]
    


interval_undefine = list(sven([33.8, 33.8], [0,1], 0.1))
print('Інтервал невизначеності', interval_undefine)


### Метод золотого перетину

In [ ]:
def golden_ratio(interval, x, s, epsilon):
    """
    Функція для реалізації методу Золотого перетину
    """
    calls_count = 0
    a = min(interval)
    b = max(interval)
    done = False
    
    i = 0

    while not done:
        i += 1
        
        L = abs(a - b)
        done = True if L < epsilon else False
        l1 = a + 0.382 * L
        l2 = a + 0.618 * L

        l1_val = func(x[0] + l1 * s[0], x[1] + l1 * s[1])
        l2_val = func(x[0] + l2 * s[0], x[1] + l2 * s[1])
        
        calls_count += 2
        if l1_val < l2_val:
            b = l2
            potential_min = l1
        elif l2_val <= l1_val:
            a = l1
            potential_min = l2
    print("Кількість ітерацій -", i)
    print("Кількість визовів функції -", calls_count)
    
    return potential_min

print("Результат:", golden_ratio(interval_undefine, [33.8, 33.8], [0,1], 0.2))

### Метод ДСК-Пауелла

In [ ]:
def dsk_powell(interval, x, s, epsilon):
    """
    Функція для реалізації методу ДСК-Пауелла
    """
    calls_count = 0
    
    i = 1
    done = False

    l1 = min(interval)
    l3 = max(interval)
    l2 = (l1 + l3) * 0.5
    
    l1_val = func(x[0] + l1 * s[0], x[1] + l1 * s[1])
    l2_val = func(x[0] + l2 * s[0], x[1] + l2 * s[1])
    l3_val = func(x[0] + l3 * s[0], x[1] + l2 * s[1])
    calls_count += 3
    
    la = l2 + (l2 - l1) * (l1_val - l3_val) / 2 / (l1_val - 2 * l2_val + l3_val)
    la_val = func(x[0] + la * s[0], x[1] + la * s[1])
    calls_count += 1
    
    if abs(l2_val - la_val) <= epsilon and abs(l2 - la) <= epsilon:
        done = True

    while not done:
        i += 1

        left = 1 if la < l2 else 0
        min_idx = np.argmin([l2_val, la_val])
        if min_idx:
            if left:
                l3 = l2
                l3_val = l2_val
            else:
                l1 = l2
                l1_val = l2_val
            l2 = la
            l2_val = la_val
        else:
            if left:
                l1 = la
                l1_val = la_val
            else:
                l3 = la
                l3_val = la_val
        
        a1 = (l2_val - l1_val) / (l2 - l1)
        a2 = ((l3_val - l1_val) / (l3 - l1) - (l2_val - l1_val) / (l2 - l1)) / (l2 - l1)
        
        la = (l1 + l2) * 0.5 - a1 / (2 * a2)
        la_val = func(x[0] + la * s[0], x[1] + la * s[1])
        calls_count += 1
        
        if abs(la - l2) <= epsilon and abs(la_val - l2_val) <= epsilon:
            done = True

    print("Кількість ітерацій -", i)
    print("Кількість визовів функції -", calls_count)
    
    return la

print("Результат:", dsk_powell(interval_undefine, [33.8, 33.8], [0,1], 0.2))     

### Основна функція - Модифікований метод Ньютона

In [ ]:
def main(x, criterium, step_h, epsilon, method_one_dim, method_epsilon, sven_parametr, schema):
    '''
    Основна функція, що буде повертати результат залежно від умов - метода одновимірного пошуку,
    критерія закінчення, кроку при обчисленні похідних, значення епсілон (для одновимірного
    і загального випадку), параметра у методі Свена та схеми обчислення похідних -
    лівої, правої чи центральної
    '''
    done = False
    while not done:
        s = np.matmul(routet_hessian(x[0], x[1]), gradient(x[0], x[1]))
        interval = sven(x, s, sven_parametr)
        if method_one_dim == 'golden ratio':
            lambda_opt = golden_ratio(interval, x, s, method_epsilon)
        else:
            lambda_opt = dsk_powell(interval, x, s, method_epsilon)

        xk_plus_1 = x + lambda_opt * np.matmul(routet_hessian(x[0], x[1]), gradient(x[0], x[1]))
        print(xk_plus_1)

        if criterium == "1":
            done = check_criterium1(x[0], x[1], xk_plus_1[0], xk_plus_1[1], epsilon)
        elif criterium == "2":
            done = check_criterium2(xk_plus_1[0], xk_plus_1[1], epsilon)
        x = xk_plus_1


print(main([33.8, 33.8], "2", 0.2, 0.2, 'golden ratio', 0.2, 0.1, "left"))
